In [8]:
# Project Libraries
import data

# Generic Libraries
import numpy as np
import pandas as pd

In [9]:
# ----- Useful dataframes
def dataframes():
    # Informative dataframe
    df_info = pd.DataFrame({
        'bid_volume': [0.0001],
        'ask_volume': [0.0001],
        'commission': [0.000025]
    })

    # Oders dataframe
    df_orders = pd.DataFrame(columns = ['timestamp', 'order_id', 'side', 'price', 'order_amount', 'symbol'])

    # Trades dataframe
    df_trades = pd.DataFrame(columns = ['timestamp', 'order_id', 'trade_id', 'side', 'price', 'symbol', 'filled_amount', 'order_amount', 'fee'])

    # Inventory dataframe
    df_inventory = pd.DataFrame({
        'timestamp': [0],
        'base (BTC)': [5],
        'quote (USDT)': [500000]
    })

    # Rebalance dataframe
    df_rebalances = pd.DataFrame(columns = ['timestamp', 'sent', 'received', 'price', 'fee'])

    return df_info, df_orders, df_trades, df_inventory, df_rebalances


def names(name):
    if name == 'bid':
        return 'buy'
    elif name == 'ask':
        return 'sell'
    else:
        'Function Failure: names'

# Position valuation
def filled_volume(size_variation: float, price_variation: float, operation: str = None, constant_position_base: float = .0001):
    # Ask
    if operation == 'ask':
        if price_variation > 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    # Bid
    if operation == 'bid':
        if price_variation < 0:
            current_position = constant_position_base
        elif price_variation == 0:
            if size_variation > 0:
                current_position = 0
            else:
                if size_variation <= constant_position_base:
                    current_position = size_variation
                else:
                    current_position = constant_position_base
        else:
            current_position = 0
    return current_position

def execute_trade(df_orders,side,passed_index, current_id, traded_volume, trade_fee,i, first_input:bool = False):
    if first_input == True:
        df_output = pd.DataFrame([
        df_bitfinex_tob.iloc[1]['timestamp'],
        df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_id'],
        1, side_filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 'BTC/USDT',
        filled, df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['order_amount'],
        platform_fee * filled * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']
    ], index = ['timestamp','order_id','trade_id','side','price','symbol','filled_amount','order_amount','fee']
    ).transpose()
        return df_output
    else:
        df_trades.at[passed_index,'timestamp'] = df_bitfinex_tob.iloc[i]['timestamp']
        df_trades.at[passed_index,'order_id'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_id']
        df_trades.at[passed_index,'trade_id'] = current_id
        df_trades.at[passed_index,'side'] = names(side)
        df_trades.at[passed_index,'price'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['price']
        df_trades.at[passed_index,'symbol'] = 'BTC/USDT'
        df_trades.at[passed_index,'filled_amount'] = traded_volume
        df_trades.at[passed_index,'order_amount'] = df_orders[df_orders['side'] == names(side)].iloc[-1]['order_amount']
        df_trades.at[passed_index,'fee'] = trade_fee * traded_volume * df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price']

def place_order(i, passed_index, side, current_id, amount):
    df_orders.at[passed_index, 'timestamp'] = df_bitfinex_tob.iloc[i]['timestamp']
    df_orders.at[passed_index, 'order_id'] = current_id
    df_orders.at[passed_index, 'side'] = names(side)
    df_orders.at[passed_index, 'price'] = df_bitfinex_tob.iloc[i][side]
    df_orders.at[passed_index, 'order_amount'] = amount
    df_orders.at[passed_index, 'symbol'] = 'BTC/USDT'

# ----- Inventory
def inventory(index_inventory):
    if df_trades['side'].iloc[-1] == "sell":

        df_inventory.at[index_inventory, 'timestamp'] = df_trades['timestamp'].iloc[-1]
        df_inventory.at[index_inventory, 'base (BTC)'] = df_trades['filled_amount'].iloc[-1] + df_inventory.at[index_inventory-1, 'base (BTC)']
        df_inventory.at[index_inventory, 'quote (USDT)'] = df_inventory.at[index_inventory-1, 'quote (USDT)'] - df_trades['filled_amount'].iloc[-1]*df_trades['price'].iloc[-1]

    elif df_trades['side'].iloc[-1] == "buy":

        df_inventory.at[index_inventory, 'timestamp'] = df_trades['timestamp'].iloc[-1]
        df_inventory.at[index_inventory, 'base (BTC)'] = df_inventory.at[index_inventory-1, 'base (BTC)'] - df_trades['filled_amount'].iloc[-1]
        df_inventory.at[index_inventory, 'quote (USDT)'] = df_inventory.at[index_inventory-1, 'quote (USDT)'] + df_trades['filled_amount'].iloc[-1]*df_trades['price'].iloc[-1]

In [10]:
# ----- Market information
df_bitfinex_tob = pd.DataFrame(data.read_file(file_name = "orderbooks_05jul21.json", folder_route = "files/")['bitfinex']).transpose().reset_index()
df_bitfinex_tob['index'] = pd.to_datetime(df_bitfinex_tob['index'])
df_bitfinex_tob = df_bitfinex_tob.set_index('index').resample('min').last().ffill()
for column in df_bitfinex_tob.columns:
    df_bitfinex_tob[column] = df_bitfinex_tob[column].apply(lambda x: x['0'])
df_bitfinex_tob = df_bitfinex_tob.reset_index().rename(columns = {'index':'timestamp'})

# SLA Conditions
contracted_volume = .0001
platform_fee = .000025

# Inventory DataFrmae (Initial)
df_inventory = pd.DataFrame({
    'timestamp': [0],
    'base (BTC)': [5],
    'quote (USDT)': [500000]
})

# Initial Order
df_orders = pd.DataFrame([
    [df_bitfinex_tob.iloc[0]['timestamp'], df_bitfinex_tob.iloc[0]['timestamp']],
    [1,2], ['buy','sell'], [df_bitfinex_tob.iloc[0]['bid'], df_bitfinex_tob.iloc[0]['ask']],
    [contracted_volume, contracted_volume], ['BTC/USDT','BTC/USDT']
    ], 
    index =  ['timestamp','order_id','side','price', 'order_amount','symbol'],
    columns = [0,1]
).transpose()

for side_filled in ['ask','bid']:
    filled = filled_volume(
        size_variation = df_bitfinex_tob.iloc[1][side_filled+'_size'] - df_bitfinex_tob.iloc[0][side_filled+'_size'],
        operation = side_filled, 
        price_variation = df_bitfinex_tob.iloc[1][side_filled] - df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 
        constant_position_base = contracted_volume
    )

    if filled > 0:
            df_trades = execute_trade(
                df_orders = df_orders,
                side = side_filled,
                i = 1,
                passed_index = 0, 
                current_id = 1, 
                traded_volume = filled, 
                trade_fee = platform_fee, 
                first_input = True)
            
            # ---- Liquidity Pool Adjustment
            passed_index_inventory = df_inventory.index[-1] + 1
            inventory(index_inventory = passed_index_inventory )
            # ----

            last_id = df_orders.iloc[-1]['order_id']
            place_order(
                i = 1,
                passed_index = 2,
                side = side_filled,
                current_id = last_id + 1, 
                amount = contracted_volume
                )
df_inventory


,timestamp,base (BTC),quote (USDT)
0,0.0,5.0000,500000.0000
1,2021-07-05 13:07:00+00:00,5.0001,499997.1717


In [11]:
for period in range(2,len(df_bitfinex_tob)):
    for side_filled in ['ask','bid']:
        filled = filled_volume(
            size_variation = df_bitfinex_tob.iloc[period][side_filled+'_size'] - df_bitfinex_tob.iloc[period - 1][side_filled+'_size'],
            operation = side_filled, 
            price_variation = df_bitfinex_tob.iloc[period][side_filled] - df_orders[df_orders['side'] == names(side_filled)].iloc[-1]['price'], 
            constant_position_base = contracted_volume
        )

        if filled > 0:
            last_id_trades = df_trades.iloc[-1]['trade_id']
            passed_index_trades = df_trades.index[-1] + 1
            execute_trade(
                df_orders = df_orders,
                side = side_filled,
                i = period,
                passed_index = passed_index_trades, 
                current_id = last_id_trades + 1, 
                traded_volume = filled, 
                trade_fee = platform_fee, 
                )
            
            # ---- Liquidity Pool Adjustment
            passed_index_inventory = df_inventory.index[-1] + 1
            inventory(index_inventory = passed_index_inventory )
            # ----

            last_id_orders = df_orders.iloc[-1]['order_id']
            passed_index_orders = df_orders.index[-1] + 1
            place_order(
                i = period,
                passed_index = passed_index_orders,
                side = side_filled,
                current_id = last_id_orders + 1, 
                amount = contracted_volume
                )
df_inventory

,timestamp,base (BTC),quote (USDT)
0,0.0,5.0000,500000.0000
1,2021-07-05 13:07:00+00:00,5.0001,499997.1717


In [14]:
df_inventory

,timestamp,base (BTC),quote (USDT)
0,0.0,5.0000,500000.0000
1,2021-07-05 13:07:00+00:00,5.0001,499997.1717
